# 📊 LLM Post-Filter Experiment: Data Extraction

**Focus**: Extract GLITCH detections from baseline experiments for LLM evaluation.

## 🎯 Goals:
1. Extract all TP/FP GLITCH detections from Chef and Puppet experiments
2. Prepare clean dataset for LLM post-filtering pipeline
3. Generate summary statistics for baseline performance

## 📁 Data Sources:
- **Baseline Results**: Chef and Puppet static analysis experiments
- **Target Smells**: Hard-coded secret, Suspicious comment, Weak cryptography


## 🔧 Setup and Imports


In [1]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path

# Add the src directory to path to import our llm_postfilter modules
project_root = Path.cwd().parent.parent.parent
sys.path.append(str(project_root / "src"))

from llm_postfilter.data_extractor import GLITCHDetectionExtractor

print(f"Project root: {project_root}")
print(f"Working directory: {Path.cwd()}")

# Initialize the extractor
extractor = GLITCHDetectionExtractor(project_root)
print("✅ Data extractor initialized successfully!")

Project root: /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval
Working directory: /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/notebooks
✅ Data extractor initialized successfully!


## 📊 Extract Chef Detections


In [2]:
print("🔍 Extracting Chef detections...")

chef_detections = extractor.extract_detections_for_llm('chef')

print(f"\n📊 Chef Detection Summary:")
for smell, detections in chef_detections.items():
    tp_count = sum(1 for d in detections if d['is_true_positive'])
    fp_count = sum(1 for d in detections if not d['is_true_positive'])
    print(f"  {smell}: {len(detections)} total | {tp_count} TP | {fp_count} FP")

# Show example detection structure
if chef_detections:
    first_smell = next(iter(chef_detections.keys()))
    if chef_detections[first_smell]:
        print(f"\n📝 Example detection structure:")
        example = chef_detections[first_smell][0]
        for key, value in example.items():
            print(f"  {key}: {value}")
    else:
        print(f"\n⚠️  No detections found for {first_smell}")
else:
    print("\n❌ No detections extracted!")

INFO:llm_postfilter.data_extractor:Loaded chef data: Oracle=148, GLITCH=166
INFO:llm_postfilter.data_extractor:Hard-coded secret: 9 TP, 37 FP
INFO:llm_postfilter.data_extractor:Suspicious comment: 4 TP, 6 FP
INFO:llm_postfilter.data_extractor:Use of weak cryptography algorithms: 1 TP, 1 FP
INFO:llm_postfilter.data_extractor:Extracted 46 detections for Hard-coded secret
INFO:llm_postfilter.data_extractor:Extracted 10 detections for Suspicious comment
INFO:llm_postfilter.data_extractor:Extracted 2 detections for Use of weak cryptography algorithms


🔍 Extracting Chef detections...

📊 Chef Detection Summary:
  Hard-coded secret: 46 total | 9 TP | 37 FP
  Suspicious comment: 10 total | 4 TP | 6 FP
  Use of weak cryptography algorithms: 2 total | 1 TP | 1 FP

📝 Example detection structure:
  detection_id: chef_Hard-coded secret_chef-boneyard_qa-chef-server-cluster-attributes-default.rb_3
  iac_tool: chef
  smell_category: Hard-coded secret
  glitch_smell: hardcoded-secret
  file_path: chef-boneyard_qa-chef-server-cluster-attributes-default.rb
  line_number: 3
  detection_id_raw: chef-boneyard_qa-chef-server-cluster-attributes-default.rb_3
  is_true_positive: True
  glitch_detection: True


## 📊 Extract Puppet Detections


In [3]:
print("🔍 Extracting Puppet detections...")

puppet_detections = extractor.extract_detections_for_llm('puppet')

print(f"\n📊 Puppet Detection Summary:")
for smell, detections in puppet_detections.items():
    tp_count = sum(1 for d in detections if d['is_true_positive'])
    fp_count = sum(1 for d in detections if not d['is_true_positive'])
    print(f"  {smell}: {len(detections)} total | {tp_count} TP | {fp_count} FP")

# Cross-tool comparison
print(f"\n🔄 Chef vs Puppet Comparison:")
all_smells = set(chef_detections.keys()) | set(puppet_detections.keys())
for smell in sorted(all_smells):
    chef_count = len(chef_detections.get(smell, []))
    puppet_count = len(puppet_detections.get(smell, []))
    print(f"  {smell}: Chef={chef_count}, Puppet={puppet_count}")

total_detections = (sum(len(detections) for detections in chef_detections.values()) + 
                   sum(len(detections) for detections in puppet_detections.values()))
print(f"\n✅ Total detections extracted: {total_detections}")

INFO:llm_postfilter.data_extractor:Loaded puppet data: Oracle=117, GLITCH=197
INFO:llm_postfilter.data_extractor:Hard-coded secret: 9 TP, 57 FP
INFO:llm_postfilter.data_extractor:Suspicious comment: 9 TP, 14 FP
INFO:llm_postfilter.data_extractor:Use of weak cryptography algorithms: 4 TP, 3 FP
INFO:llm_postfilter.data_extractor:Extracted 66 detections for Hard-coded secret
INFO:llm_postfilter.data_extractor:Extracted 23 detections for Suspicious comment
INFO:llm_postfilter.data_extractor:Extracted 7 detections for Use of weak cryptography algorithms


🔍 Extracting Puppet detections...

📊 Puppet Detection Summary:
  Hard-coded secret: 66 total | 9 TP | 57 FP
  Suspicious comment: 23 total | 9 TP | 14 FP
  Use of weak cryptography algorithms: 7 total | 4 TP | 3 FP

🔄 Chef vs Puppet Comparison:
  Hard-coded secret: Chef=46, Puppet=66
  Suspicious comment: Chef=10, Puppet=23
  Use of weak cryptography algorithms: Chef=2, Puppet=7

✅ Total detections extracted: 154


## 💾 Save Detection Dataset


In [4]:
print("💾 Saving detection dataset...")

output_dir = project_root / "experiments/llm_postfilter/data"
output_dir.mkdir(parents=True, exist_ok=True)

# Save detection files
chef_saved = extractor.save_detections('chef', output_dir)
puppet_saved = extractor.save_detections('puppet', output_dir)

print(f"✅ Detection files saved:")
for file_path in sorted(output_dir.glob("*_detections.csv")):
    file_size = file_path.stat().st_size
    df = pd.read_csv(file_path)
    print(f"  📄 {file_path.name}: {len(df)} detections ({file_size:,} bytes)")

print(f"\n🎯 Dataset ready for LLM post-filtering pipeline!")
print(f"📁 Location: {output_dir}")

INFO:llm_postfilter.data_extractor:Loaded chef data: Oracle=148, GLITCH=166
INFO:llm_postfilter.data_extractor:Hard-coded secret: 9 TP, 37 FP
INFO:llm_postfilter.data_extractor:Suspicious comment: 4 TP, 6 FP
INFO:llm_postfilter.data_extractor:Use of weak cryptography algorithms: 1 TP, 1 FP
INFO:llm_postfilter.data_extractor:Extracted 46 detections for Hard-coded secret
INFO:llm_postfilter.data_extractor:Extracted 10 detections for Suspicious comment
INFO:llm_postfilter.data_extractor:Extracted 2 detections for Use of weak cryptography algorithms
INFO:llm_postfilter.data_extractor:Saved 46 detections to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/data/chef_hard_coded_secret_detections.csv
INFO:llm_postfilter.data_extractor:Saved 10 detections to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiment

💾 Saving detection dataset...
✅ Detection files saved:
  📄 chef_hard_coded_secret_detections.csv: 46 detections (12,068 bytes)
  📄 chef_suspicious_comment_detections.csv: 10 detections (3,176 bytes)
  📄 chef_use_of_weak_cryptography_algorithms_detections.csv: 2 detections (795 bytes)
  📄 puppet_hard_coded_secret_detections.csv: 66 detections (18,591 bytes)
  📄 puppet_suspicious_comment_detections.csv: 23 detections (6,281 bytes)
  📄 puppet_use_of_weak_cryptography_algorithms_detections.csv: 7 detections (2,504 bytes)

🎯 Dataset ready for LLM post-filtering pipeline!
📁 Location: /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/data


## 🔍 Extract Code Context for LLM Analysis

**Extract ±3 lines of code context around each GLITCH detection for LLM evaluation.**

This provides transparent, analyzable code snippets for the LLM post-filtering pipeline.


In [5]:
# Import context extractor
from llm_postfilter.context_extractor import CodeContextExtractor

# Setup directories
context_dir = output_dir / "with_context"
context_dir.mkdir(exist_ok=True)

# Initialize context extractor
context_extractor = CodeContextExtractor(project_root)

# Find detection files to process
detection_files = list(output_dir.glob("*_detections.csv"))
detection_files = [f for f in detection_files if not f.name.endswith("_with_context.csv") and not f.name.endswith("_llm_filtered.csv")]

print(f"📁 Found {len(detection_files)} detection files:")
for file in detection_files:
    df = pd.read_csv(file)
    tp_count = df['is_true_positive'].sum()
    fp_count = len(df) - tp_count
    print(f"  📄 {file.name}: {len(df)} detections ({tp_count} TP, {fp_count} FP)")

print(f"\n🔍 Extracting code context for LLM analysis...")

📁 Found 6 detection files:
  📄 puppet_hard_coded_secret_detections.csv: 66 detections (9 TP, 57 FP)
  📄 puppet_use_of_weak_cryptography_algorithms_detections.csv: 7 detections (4 TP, 3 FP)
  📄 puppet_suspicious_comment_detections.csv: 23 detections (9 TP, 14 FP)
  📄 chef_suspicious_comment_detections.csv: 10 detections (4 TP, 6 FP)
  📄 chef_use_of_weak_cryptography_algorithms_detections.csv: 2 detections (1 TP, 1 FP)
  📄 chef_hard_coded_secret_detections.csv: 46 detections (9 TP, 37 FP)

🔍 Extracting code context for LLM analysis...


In [6]:
# Process each detection file and save context-enhanced versions
context_enhanced_files = []
context_stats = []

for i, detection_file in enumerate(detection_files):
    print(f"\n🔄 Processing {i+1}/{len(detection_files)}: {detection_file.name}")
    
    # Extract context and save enhanced file
    enhanced_df = context_extractor.process_and_save_detections(
        detection_file, context_dir, context_lines=3
    )
    
    # Track files and stats
    base_name = detection_file.stem
    context_file = context_dir / f"{base_name}_with_context.csv"
    context_enhanced_files.append(context_file)
    
    # Calculate stats
    total_detections = len(enhanced_df)
    successful_context = enhanced_df['context_success'].sum()
    files_found = enhanced_df['file_found'].sum()
    
    context_stats.append({
        'file': detection_file.name,
        'total_detections': total_detections,
        'files_found': files_found,
        'context_extracted': successful_context,
        'success_rate': successful_context / total_detections if total_detections > 0 else 0
    })
    
    print(f"✅ {context_file.name}: {successful_context}/{total_detections} context extracted ({successful_context/total_detections:.1%})")

print(f"\n🎯 Context extraction completed for {len(detection_files)} files!")

INFO:llm_postfilter.context_extractor:Loaded 66 detections from puppet_hard_coded_secret_detections.csv
INFO:llm_postfilter.context_extractor:Successfully extracted context for 66/66 detections
INFO:llm_postfilter.context_extractor:Saved 66 detections with context to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/data/with_context/puppet_hard_coded_secret_detections_with_context.csv
INFO:llm_postfilter.context_extractor:Context extraction success rate: 66/66 (100.0%)
INFO:llm_postfilter.context_extractor:Loaded 7 detections from puppet_use_of_weak_cryptography_algorithms_detections.csv
INFO:llm_postfilter.context_extractor:Successfully extracted context for 7/7 detections
INFO:llm_postfilter.context_extractor:Saved 7 detections with context to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/ll


🔄 Processing 1/6: puppet_hard_coded_secret_detections.csv
✅ puppet_hard_coded_secret_detections_with_context.csv: 66/66 context extracted (100.0%)

🔄 Processing 2/6: puppet_use_of_weak_cryptography_algorithms_detections.csv
✅ puppet_use_of_weak_cryptography_algorithms_detections_with_context.csv: 7/7 context extracted (100.0%)

🔄 Processing 3/6: puppet_suspicious_comment_detections.csv
✅ puppet_suspicious_comment_detections_with_context.csv: 23/23 context extracted (100.0%)

🔄 Processing 4/6: chef_suspicious_comment_detections.csv
✅ chef_suspicious_comment_detections_with_context.csv: 10/10 context extracted (100.0%)

🔄 Processing 5/6: chef_use_of_weak_cryptography_algorithms_detections.csv
✅ chef_use_of_weak_cryptography_algorithms_detections_with_context.csv: 2/2 context extracted (100.0%)

🔄 Processing 6/6: chef_hard_coded_secret_detections.csv
✅ chef_hard_coded_secret_detections_with_context.csv: 46/46 context extracted (100.0%)

🎯 Context extraction completed for 6 files!


In [7]:
# Display context extraction summary
print("📊 Context Extraction Summary")
print("=" * 50)

stats_df = pd.DataFrame(context_stats)
for _, row in stats_df.iterrows():
    print(f"📄 {row['file']}: {row['context_extracted']}/{row['total_detections']} ({row['success_rate']:.1%}) context extracted")

# Overall statistics
total_detections = stats_df['total_detections'].sum()
total_context_extracted = stats_df['context_extracted'].sum()
overall_success_rate = total_context_extracted / total_detections if total_detections > 0 else 0

print(f"\n🎯 Overall: {total_context_extracted}/{total_detections} ({overall_success_rate:.1%}) successful")
print(f"📁 Context files saved: {len(context_enhanced_files)} → {context_dir}")

📊 Context Extraction Summary
📄 puppet_hard_coded_secret_detections.csv: 66/66 (100.0%) context extracted
📄 puppet_use_of_weak_cryptography_algorithms_detections.csv: 7/7 (100.0%) context extracted
📄 puppet_suspicious_comment_detections.csv: 23/23 (100.0%) context extracted
📄 chef_suspicious_comment_detections.csv: 10/10 (100.0%) context extracted
📄 chef_use_of_weak_cryptography_algorithms_detections.csv: 2/2 (100.0%) context extracted
📄 chef_hard_coded_secret_detections.csv: 46/46 (100.0%) context extracted

🎯 Overall: 154/154 (100.0%) successful
📁 Context files saved: 6 → /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/data/with_context


In [8]:
# Show example context snippet that LLM will analyze
print("🔍 Example Context Snippet for LLM")
print("=" * 40)

if context_enhanced_files:
    example_file = context_enhanced_files[0]
    example_df = pd.read_csv(example_file)
    successful_detections = example_df[example_df['context_success'] == True]
    
    if len(successful_detections) > 0:
        example = successful_detections.iloc[0]
        print(f"📁 {example_file.name}")
        print(f"🎯 {example['smell_category']} | TP: {example['is_true_positive']}")
        print(f"\n📄 Context Snippet:")
        print("-" * 30)
        print(example['context_snippet'])
        print("-" * 30)
    else:
        print("❌ No successful context extractions found")
else:
    print("❌ No context-enhanced files available")

print(f"\n💡 All context files saved → {context_dir}")

print(f"\n🎯 Data preparation completed!")
print(f"📁 Detection files: {output_dir}")
print(f"📁 Context files: {context_dir}")
print(f"\n➡️  Next: Run 02_llm_experiment.py for LLM evaluation")



🔍 Example Context Snippet for LLM
📁 puppet_hard_coded_secret_detections_with_context.csv
🎯 Hard-coded secret | TP: True

📄 Context Snippet:
------------------------------
# File: alphagov@govuk-puppet-modules-users-manifests-felisialoukou.pp
# Target line: 5

      2: class users::felisialoukou { govuk_user { 'felisialoukou':
      3:     fullname => 'Felisia Loukou',
      4:     email    => 'felisia.loukou@digital.cabinet-office.gov.uk',
>>>   5:     ssh_key  => [
      6:         'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDlxJNWiWOVbtSwe5DNNzN8csunIvCX6XUIAT141EjGZpOaEjK3yFtvv96OukdWTXPQXnWBDOIWg+fNJWc4LsfqnCV5CrHftykTqHeKakEcVX2aW5UrWvlOWHoTdkbFv+L67MaT2xIT/KWLYPliyZJOmwwF+W1kiFq3xVtI5qgMha7s2I3thuBo8lgLXsmxdVcYoa32MBKabNyknvZaF/l2l/D/wssbC/3N5zeLupJJXDRA4BWw6nqAW97AOceVH04twCG6+B5qL/M56D/YW8kMIx/XSSIAde4+yXCX0gLr5p4L6pNvD2vn/nJSY/rMDtO5rPX+XQjmM7eti50FvkflvGzuaUoSXb1sP+rwOtGMVQBn4pnB1whd8jZSQC+SRsXAgj7TgQfKLux/T00cq+pC09HrBPLag1TUcueJE65FM6mi2uyi+sYYOnVfr4ZhgiEBY+uFoOWl/TwnKuIAXXXD4aL